## Due date: 5/1/2025
### Nandini Doddi & Jeslyn Miranda
For your group project, you are requested to do a complete process of machine learning, which includes
Feature extraction, classification, and analysis.
You can access the data that we are working with on Canvas -> Modules =-> Subcellular Data

### 1:HIIII You need to get the protein sequences (as it was discussed in assignment 3) and extract relevant
features that are representative of the protein sequences. You can also use the features that are already
available (e.g., Occurrence and Composition).

###Project Goal
Predict the subcellular localization of Gram-positive bacterial proteins based on their sequences. It’s a multi-class classification problem (4 classes = 4 possible cell locations).

✅ Step-by-Step Plan
Step 1: Feature Extraction
You can use:

✅ Amino Acid Composition (AAC): % of each amino acid in the protein

✅ Dipeptide Composition (optional if you want deeper features)

✅ Occurrence Features (e.g., binary vector showing whether a certain amino acid appears)

✅ Pre-extracted features if provided (data1, data2, etc.)
Ex:
from collections import Counter

def get_aac(sequence):
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
    count = Counter(sequence)
    total = len(sequence)
    return [count[aa]/total for aa in amino_acids]


In [1]:
from collections import Counter
# imports Python's built-in Counter class, which helps count how many times each letter appears in a sequence.
#We'll use it to count how often each amino acid appears in a sequence

amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
#creates a string of all 20 standard amino acids, represented by each letter
#this ensures we always calculate features in the same order- even if some acids don't appear in the sequence

def compute_aac(sequence): #function to calculate amino acid composition (AAC) for one sequence
    length = len(sequence) #measures how long the protein sequence is, or how many letters long
    counts = Counter(sequence) #creates a dictionary-like object that counts how many times each acid appears
    #ex: for "AAG", counts = {'A": 2, 'G' : 1}
    return [counts[aa] / length for aa in amino_acids] #creates a list of relative frequencies- for each amino acid (aa), it divides the count by the sequence length
    #ensures each feature is normalized so sequences of different lengths are comparable
    #Output: A list of 20 numbers, one for each aa

In [6]:
## Now apply the function to your dataset
import pandas as pd

#load the dataset
g_data = pd.read_csv('g_data (1).csv', header=None)
#loads the gram=positive data from the file into a pandas dataframe

### 2: HEY You will need to prepare the data (properly put them together with labels) and use different machine
learning methods on them (e.g., K-Nearest Neighbor (KNN), Support Vector Machine (SVM), Naïve
Bayes, Artificial Neural Network (ANN), Random Forest, Bagging). As the data suggest, the problem is
hand is a multi-class classification task.
Note: The problem is protein subcellular localization. The proteins can function in different locations in
the cell. Given a protein, it is important to identify its functioning location. For this problem, we are
working with Gram-positive bacterial proteins, which function in 4 locations in the cell. So, the problem
is to get the training data, extract relevant features, and build a model to predict the functioning
location of a given protein.

Prepare Data
Merge feature files and add labels (Fold 1, Fold 2, etc.).

Ensure labels are correctly encoded for classification (e.g., use LabelEncoder).

Train/test split or k-fold for evaluation.



### 3: You then need to analyze and interpret your output using different approaches such as:
3.1: Using independent test set
3.2: k-fold cross validation
3.3: check different accuracy measurement
3.4: Discuss your results and interpret the output
Note: You can do steps 2 and 3 in Python programming language.

tep 3: Apply ML Models
Use scikit-learn (or Keras for neural networks):

Example classifiers:
python
Copy code
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

model = RandomForestClassifier()
model.fit(X_train, y_train)
print(classification_report(y_test, model.predict(X_test)))

Also try:

KNeighborsClassifier

SVC

GaussianNB

MLPClassifier (ANN)

BaggingClassifier

### As the output: You will prepare a report (in Word or text document) that explains what you did in each
and every step in detail and present your results, analysis, and discussion (no more than 6 pages). You
will also send your codes with proper comments along with your report.

6-page max report in Word/Text format:

Intro → Feature Engineering → ML Methods → Evaluation → Conclusion

Code: Well-commented .py or .ipynb files